In [1]:
import numpy as np
import pandas as pd

In [26]:
# dict형의 key는 칼럼 value의 리스트가 행의 값들
raw_data = {'name': ['kevin','sally','hoyeon','lux'],
           'height':[178.2, 162.9, 160.6, 156.2],
           'gender':['male','female','female',None]}
pd_data = pd.DataFrame(raw_data)
pd_data.head()

,name,height,gender
0,kevin,178.2,male
1,sally,162.9,female
2,hoyeon,160.6,female
3,lux,156.2,None


In [27]:
# 결측치 전처리

# df.dropna() 사용
filtered_data = pd_data.dropna()
filtered_data.head()

# del(df['칼럼']) 사용
del(filtered_data['name'])
filtered_data.head()


,height,gender
0,178.2,male
1,162.9,female
2,160.6,female


In [28]:
# 리스트 자료 + 해서 이어붙일 수 있음
# 데이터프레임 .append하면 rbind처럼 행으로 붙일 수 있음

# 무작위로 데이터 생성하기 위해 성별 키 평균값 출력
female_mean = np.average(filtered_data[filtered_data['gender'] == 'female']['height'].values)
male_mean = np.average(filtered_data[filtered_data['gender'] == 'male']['height'].values)
print(female_mean, male_mean)

161.75 178.2


In [29]:
#난수 생성을 위한 시드(seed)설정. 인자로는 0과 같거나 큰 정수를 넣어줌
np.random.seed(0)

variance = 3
female_heights = variance * np.random.randn(200) + female_mean
male_heights = variance * np.random.randn(200) + male_mean

# 만든 array의 1~10번째까지 보기
print(female_heights[:10])

[167.04215704 162.95047163 164.68621395 168.4726796  167.35267397
 158.81816636 164.60026525 161.29592838 161.44034344 162.98179551]


In [30]:
#np.random.randn(200)

In [31]:
# 난수로 생성한 데이터 dict형태로 만들기
generated_data = {'gender':['female'] * 200 + ['male'] * 200, 'height': list(female_heights)+list(male_heights)}

# dict형태를 데이터프레임화시킨후 기존 데이터프레임에 append하기
# ignore_index=False : 인덱스 레이블을 사용하지 않겠다라는 뜻
filtered_data = filtered_data.append(pd.DataFrame(generated_data), ignore_index=True, sort=False)
filtered_data.head()

,height,gender
0,178.200000,male
1,162.900000,female
2,160.600000,female
3,167.042157,female
4,162.950472,female


In [32]:
filtered_data.shape

(403, 2)

In [33]:
# 성별을 0,1 값으로 바꿔주기
from sklearn import preprocessing

le_gender = preprocessing.LabelEncoder()
final_data = filtered_data.copy()
final_data['gender'] = le_gender.fit_transform(filtered_data['gender'])

final_data.tail()

,height,gender
398,175.915523,1
399,180.773772,1
400,181.623306,1
401,182.599736,1
402,180.757656,1


In [60]:
# 분류임에도 불구하고 linear Regression 써보기
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

#사용할 feature
features = ['height']

# k-fold cross validation
# shuffle : 마구섞는다
kf = KFold(n_splits=5, shuffle=True)

# 모델 정확도 담을 리스트
accrs = []
# k-fold 몇번 했는지 출력하기 위한 idx
fold_idx = 1

# 만든 데이터에서 train / test 데이터를 k-fold이용해서 분할
for train_idx, test_idx in kf.split(final_data):
    print(f"Fold 횟수 : {fold_idx}")
    # train, test 데이터 할당
    # iloc[index번호] 의 해당 행만 갖고옴
    train_d, test_d = final_data.iloc[train_idx], final_data.iloc[test_idx]
    
    # 예측값 설정
    train_y = train_d['gender']
    # x값 설정
    train_x = train_d[features]
    
    test_y = test_d['gender']
    test_x = test_d[features]
    
    # 모델정의
    model = LinearRegression()
    # fit함수이용해 모델 학습
    model.fit(train_x, train_y)
    
    # 정확도 score함수로 측정
    mean_accr = model.score(test_x, test_y)
    # k-fold 5번 측정한거 정확도 담는 리스트에 담기
    accrs.append(mean_accr)
    
    # k-fold횟수 출력
    fold_idx += 1

# 5번 한 정확도 평균치 출력
print(np.average(accrs))

Fold 횟수 : 1
Fold 횟수 : 2
Fold 횟수 : 3
Fold 횟수 : 4
Fold 횟수 : 5
0.8759028111912007


In [63]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

#사용할 feature
features = ['height']

# k-fold cross validation
# shuffle : 마구섞는다
kf = KFold(n_splits=5, shuffle=True)

# 모델 정확도 담을 리스트
accrs = []
# k-fold 몇번 했는지 출력하기 위한 idx
fold_idx = 1

# 만든 데이터에서 train / test 데이터를 k-fold이용해서 분할
for train_idx, test_idx in kf.split(final_data):
    print(f"Fold 횟수 : {fold_idx}")
    # train, test 데이터 할당
    # iloc[index번호] 의 해당 행만 갖고옴
    train_d, test_d = final_data.iloc[train_idx], final_data.iloc[test_idx]
    
    # 예측값 설정
    train_y = train_d['gender']
    # x값 설정
    train_x = train_d[features]
    
    test_y = test_d['gender']
    test_x = test_d[features]
    
    # 모델정의 - lbgfs = Limited-memory BFGS라는 최적화 알고리즘 사용해보기
    model = LogisticRegression(solver='lbfgs')
    # fit함수이용해 모델 학습
    model.fit(train_x, train_y)
    
    # 정확도 score함수로 측정
    mean_accr = model.score(test_x, test_y)
    # k-fold 5번 측정한거 정확도 담는 리스트에 담기
    accrs.append(mean_accr)
    
    # k-fold횟수 출력
    fold_idx += 1

# 5번 한 정확도 평균치 출력
print(np.average(accrs))

Fold 횟수 : 1
Fold 횟수 : 2
Fold 횟수 : 3
Fold 횟수 : 4
Fold 횟수 : 5
1.0


- 로지스틱이 분류가 아니라 리그레션 모델임에도 불구하고 분류에 많이 이용되는 이유는? 
    * 확률값(0~1사이값)으로 나오기 때문에
    
- 로지스틱에서 feature 정규화를 필요로하지 않는다. 이번 실습에선 정규화를 진행하지 않았는데도 좋게나왔다. 하지만 feature 정규화(normalization)이 필요할 때는 언제일까?
    * Regularization(L2 norm, L1norm) 을 적용시에 필요
    * 즉, Regularization이 필요할 때만 feature normalization필요!
    * 예를들어, 각 데이터간에 feature값들간의 차이가 너무 크다면...!
    
- 결측지 대체 대안법
    * 다른 데이터들의 평균값으로 추산
    * matrix completion
    * impute 알고리즘
    
- k-fold cross validation 자체가 k번하니까 한번만돌리면될까? 
    * 아니다. shuffle을 하게되면 매번 실험결과가 달라져서 k-fold 자체를 여러번해서 그것들의 평균값을 취해주는 것이 좋다

- 기타 실험 성능 측정 지표
    * 분류 : F1 score, recall 등등
    * 회귀 : RMSE
    
**``feature normalization시 학습/테스트 데이터에 대해 구분해서 사용할 것!!!!`` 학습데이터에서 도출해서 feature normalization 할것!! 테스트데이터에서 나온 값을 쓰면 안됌!!** 